In [2]:
import time
import os
import shutil
from itertools import islice
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import findspark
findspark.init()

from google.cloud import storage

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.memory", "16g") \
    .config("spark.driver.maxResultSize", "16g") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "4") \
    .config("spark.dynamicAllocation.maxExecutors", "100") \
    .getOrCreate()



spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
spark.conf.set("spark.sql.repl.eagerEval.maxCharsPerCell", 200)

print(spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 01:01:42 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/12/09 01:01:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/12/09 01:01:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/09 01:01:43 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
24/12/09 01:01:45 WARN org.apache.spark.util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
24/12/09 01:01:50 WARN org.apache.spark.util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
24/12/09 01:01:52 WARN org.apache.spark.util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, plea

3.1.3


# Read from folder into spark df

You will see 5 sub-folders, each containing a collection of parquet files.  A single folder can be read into Spark Dataframe:

* Commits (gs://msca-bdp-data-open/final_project_git/commits): This contains information about the commits made to repositories. Each commit has metadata such as the author, committer, commit date, SHA, parent commit(s), and commit message.

* Contents (gs://msca-bdp-data-open/final_project_git/contents): Provides the content of the files in the repositories. This is useful if you're looking to analyze source code or documents within repositories.

* Files (gs://msca-bdp-data-open/final_project_git/files): This contains metadata about the files in the repositories such as the file path, the mode, and the blob ID which links back to the content.

* Languages (gs://msca-bdp-data-open/final_project_git/languages): Each repository often has code written in one or more languages. This table provides an aggregation of the number of bytes of code for each language in a repository.

* Licenses (gs://msca-bdp-data-open/final_project_git/licenses): Contains information on the licenses used by repositories.
 

In [8]:
#!hadoop fs -ls "gs://msca-bdp-data-open/final_project_git/commits"

In [9]:
# Read into spark df
commits_spDf = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/commits")
contents_spDf = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/contents")
files_spDf = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/files")
languages_spDf = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/languages")
licenses_spDf = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/licenses")

In [ ]:
# Checking preliminary data
commits_spDf.limit(5)

# EDA
What is the timeline of the data?  Do you see significant peaks and valleys?

Do you see any data collection gaps?

Do you see any outliers?  Remove obvious outliers before plotting the timeline

Do you see any spikes?  Are these spikes caused by real activities / events?